### Extract text from Pdf documents and apply NER

In [1]:
import pdfplumber, os
import os.path as osp
import pandas as pd
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from pdf_extract.utils import utils
from importlib import reload
import fasttext

reload(glob)
reload(config)
reload(utils)
reload(file)

Environment Variable: UC_CODE_DIR has been set to default: /home/alexv84/Documents/GitHub/pdf_extract/src
Environment Variable: UC_DATA_DIR has been set to default: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data
Environment Variable: UC_LANG_ID has been set to default: /home/alexv84/Documents/Arbeit/Allianz/AZVers/fasttext_langdetect
Environment Variable: UC_DATA_PKG_DIR has been set to default: 
Environment Variable: UC_DB_CONNECTION has been set to default: postgresql://postgres...
Environment Variable: UC_PORT has been set to default: 5000
Environment Variable: UC_APP_CONNECTION has been set to default: 127.0.0.1


<module 'pdf_extract.services.file' from '/home/alexv84/Documents/GitHub/pdf_extract/src/pdf_extract/services/file.py'>

In [2]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)


12
13


Read in all Pdfs in directory:

Postives only:

In [5]:
raw_corpus1 = pd.DataFrame(columns=['text', 'label'])

for z, fname in enumerate(list_of_POS_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = text  
raw_corpus1['label'] = 'positive'

Negatives only:

In [6]:
raw_corpus2 = pd.DataFrame(columns=['text','label'])

for z, fname in enumerate(list_of_NEG_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus2.loc[z] = text   
raw_corpus2['label'] = 'negative'

In [9]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)

In [8]:
#page_objects

#page.extract_words()

In [9]:
#page_instance = page_objects['1']

In [38]:
#pdf.metadata
#page_instance

Preprocess text corpus:

In [10]:
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = False)

X = raw_corpus['text']

X_cl = cleaner.fit_transform(X)
#docs = X_cl.tolist()    

Using english language.
Using 179 stop words.
Added 0 stopword(s).
Adding custom stop words...
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
raw_corpus['text_cleaned'] = X_cl
X = raw_corpus[['label', 'text_cleaned']].reset_index(drop=True)
#X.head()

In [12]:
from sklearn.model_selection import train_test_split

reload(utils)

#X_train, X_valid, X_test, y_train, y_valid, y_test = utils.train_test_split_extend(X['text_cleaned'], y=X['label'], test_size=[0.2, 0.2], random_state=42, shuffle=True)
X_train, X_test, y_train, y_test = utils.train_test_split_extend(X['text_cleaned'], y=X['label'], test_size=[0.4], random_state=123, shuffle=True)

print(X_train.shape)
#print(y_train.shape)
print(X_test.shape)
#print(X_valid.shape)


(15,)
(10,)


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [14]:
parameters = {
    'vectorizer__max_df': (0.5, 0.75, 1.0),
    # 'vectorizer__max_features': (None, 5000, 10000, 50000),
    'vectorizer__ngram_range': ((2, 2), (1, 3), (2, 3), (3, 3), (3, 5))  # unigrams or bigrams
}

pipeline = Pipeline([
   #('cleaner', utils.clean_text(verbose=False)),
   ('vectorizer', CountVectorizer(lowercase=True, 
                       token_pattern = '(?u)(?:(?!\d)\w)+\\w+', analyzer = 'word', tokenizer = None, stop_words = None)), 
   #('scaler', StandardScaler(with_mean=False)),
   ('model', MultinomialNB())
])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)


In [15]:
gs = grid_search.fit(X_train, y_train)

#vect = pipeline.fit_transform(X['text_cleaned'], X['fasttext_label'])

#pipeline.fit(X['text_cleaned'], X['fasttext_label'])


Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [16]:
best_parameters = grid_search.best_estimator_.get_params()

# best_model refit:
trained = grid_search.best_estimator_

In [24]:
#-------------
# Save model:
#-------------
file.PickleService(path="trained_model.pkl", root_path=glob.UC_DATA_DIR, is_df=False).doWrite(X=trained)


Pickle Service Read from file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/trained_model.pkl


In [20]:
# Predict:
y_pred = trained.predict(X_train)

#print(confusion_matrix(test_set['label'], y_pred))
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         8
    positive       1.00      1.00      1.00         7

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



In [21]:
# Predict:
y_pred = trained.predict(X_test)

#print(confusion_matrix(test_set['label'], y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       1.00      0.40      0.57         5
    positive       0.62      1.00      0.77         5

    accuracy                           0.70        10
   macro avg       0.81      0.70      0.67        10
weighted avg       0.81      0.70      0.67        10



In [36]:
class_probs = pd.DataFrame(trained.predict_proba(X_test)[:,1], columns=['Prob. positive class'])
class_probs.sort_values(by = 'Prob. positive class', ascending=False).head(10)

,Prob. positive class
0,1.000000
3,0.999999
8,0.999977
6,0.999880
4,0.999499
7,0.989767
1,0.932621
9,0.878249
5,0.089161
2,0.034649
